# Chipotle Macroeconomic Analysis

# Topic Summary
This project analyzes the impact of macroeconmic factors like GDP, unemployment rate, inflation, etc on Chipotle's performance from 2015-2025. It seeks to understand how the economic health of the countries Chipotle operates in influences Chipotle's revenue, net income, and stock price. The analysis combines data from the IMF, scraped data from Eulerpool, and stock price data from Alpha Vantage API. The core goal is to quantify the relationship between the economy and Chipotle's profitabiltiy.


## Data Sources

Downloaded Dataset Source: https://data.imf.org/en/Data-Explorer?datasetUrn=IMF.RES:WEO(9.0.0)

Web Collection #1 Source: https://eulerpool.com/en/stock/Chipotle-Mexican-Grill-Stock-US1696561059

Web Collection #2 Source: https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY&symbol=CMG&apikey=YOUR_APLKEY


## Package Imports

In [28]:
import pandas as pd
import numpy as np
import requests
import re
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Data Collection and Cleaning


## Downloaded Dataset: IMF World Economic Outlook



In [29]:
def data_parser():
    # Loading the CSV data to DataFrame
    df_downloaded = pd.read_csv("raw_world_economic_outlook.csv")
    
    # Countries that Chipotle operates in
    countries_to_keep = ["United States", "Canada", "United Kingdom", "France", "Germany"] 
    df_countries = df_downloaded[df_downloaded["COUNTRY"].isin(countries_to_keep)].copy()
    
    # Filtering for relevant indicators
    indicators_list = ['Unemployment rate', 'Gross domestic product (GDP), Current prices, US dollar', 'All Items, Consumer price index (CPI), Period average', 'Gross capital formation, Percent of GDP', 'Net lending (+) / net borrowing (-), General government, Percent of GDP']
    df_countries_filtered = df_countries[df_countries['INDICATOR'].isin(indicators_list)].copy()
 
    col_names = {
    'Unemployment rate': 'Unemployment Rate (%)',
    'Gross domestic product (GDP), Current prices, US dollar': 'GDP (US dollars in billions)',
    'All Items, Consumer price index (CPI), Period average': 'Consumer price index (CPI)',
    'Gross capital formation, Percent of GDP': 'Gross capital formation (% of GDP)',
    'Net lending (+) / net borrowing (-), General government, Percent of GDP': 'Net lending/borrowing (% of GDP)'
    }
    df_countries_filtered['INDICATOR'] = df_countries_filtered['INDICATOR'].replace(col_names)

    # Keeping only relevant years and necessary columns
    years_to_keep = [str(year) for year in range(2015, 2026)]
    available_years = [year for year in years_to_keep if year in df_countries_filtered.columns]
    final_columns = ['COUNTRY', 'INDICATOR'] + available_years
    
    df_cleaned = df_countries_filtered[final_columns].copy()

    for year in available_years:
        df_cleaned[year] = df_cleaned[year].astype(float)

    df_cleaned[available_years] = df_cleaned[available_years].round(2)

    df_cleaned = df_cleaned.reset_index(drop=True)

    # Resizing GDP values to be in billions
    df_cleaned.loc[df_cleaned['INDICATOR'] == 'GDP (US dollars in billions)', available_years] /= 1000000000


    # Reshaping the DataFrame to have years as a single column
    df_melted = df_cleaned.melt(id_vars=['COUNTRY', 'INDICATOR'], var_name='YEAR', value_name='VALUE')

    # Pivoting the DataFrame to have indicators as columns
    df_final = df_melted.pivot_table(index=['COUNTRY', 'YEAR'], columns='INDICATOR', values='VALUE').reset_index()

    # Writing cleaned DataFrame to CSV
    df_final.to_csv("cleaned_world_economic_outlook.csv", index=False)

    print(df_final)

############ Function Call ############
data_parser()

INDICATOR         COUNTRY  YEAR  Consumer price index (CPI)  \
0                  Canada  2015                      126.57   
1                  Canada  2016                      128.38   
2                  Canada  2017                      130.43   
3                  Canada  2018                      133.38   
4                  Canada  2019                      135.98   
5                  Canada  2020                      136.96   
6                  Canada  2021                      141.61   
7                  Canada  2022                      151.21   
8                  Canada  2023                      157.08   
9                  Canada  2024                      160.85   
10                 Canada  2025                      164.04   
11                 France  2015                      100.00   
12                 France  2016                      100.30   
13                 France  2017                      101.47   
14                 France  2018                      10

## Web Collection Dataset: Eulerpool Chipotle Analysis


In [30]:
def web_parser():
    # Retrieving HTML content
    url = "https://eulerpool.com/en/stock/Chipotle-Mexican-Grill-Stock-US1696561059"
    resp = requests.get(url)
    
    if resp.status_code != 200:
        print("Error: Unable to fetch data")
        return
    
    soup = BeautifulSoup(resp.text, "html.parser")

    # Parsing HTML content
    table = soup.find("div", {"class":"flex flex-nowrap md:mr-8 lg:mr-0 z-50"})
    years = table.find_all("span", {"class": "w-14 pr-5 whitespace-nowrap"})
    
    # Processing Header
    years_list = [year.text.strip() for year in years]
    years_list = years_list[15:26]
    years_list = [re.sub(r'\D', '', year) for year in years_list] # INCONSISTENCY: Remove non-digit characters from header ('e' in '2025e')
    years_list = [int(year) for year in years_list]
    
    # Processing Data Rows
    data_table = table.find_all("table")[1]
    data_rows = data_table.find("tbody").find_all("tr")

    revenue_row_cells = data_rows[0].find_all("td")
    all_revenue_values = [float(cell.text.strip()) for cell in revenue_row_cells]
    final_revenue_values = all_revenue_values[15:26]

    revenue_growth_row_cells = data_rows[1].find_all("td")
    all_revenue_growth_values = [float(cell.text.strip()) if cell.text.strip() not in ['-'] else 0.0 for cell in revenue_growth_row_cells]
    final_revenue_growth_values = all_revenue_growth_values[15:26]

    net_income_row_cells = data_rows[4].find_all("td")
    all_net_income_values = [float(cell.text.strip()) for cell in net_income_row_cells]   
    final_net_income_values = all_net_income_values[15:26]

    net_income_growth_row_cells = data_rows[5].find_all("td")
    all_net_income_growth_values = [float(cell.text.strip()) if cell.text.strip() not in ['-'] else 0.0 for cell in net_income_growth_row_cells] # INCONSISTENCY: Replacing with 0.0 fixes NaN value for Net Income Growth, since there was no growth
    final_net_income_growth_values = all_net_income_growth_values[15:26]

    # Creating DataFrame with years as rows and metrics as columns
    df = pd.DataFrame({
        'Year': years_list,
        'Revenue (Billions)': final_revenue_values,
        'Revenue Growth (%)': final_revenue_growth_values,
        'Net Income (Billions)': final_net_income_values,
        'Net Income Growth (%)': final_net_income_growth_values
    })

    

    # Writing cleaned DataFrame to CSV
    df.to_csv("cleaned_html.csv", index=False)

    print(df)
    
############ Function Call ############
web_parser()

    Year  Revenue (Billions)  Revenue Growth (%)  Net Income (Billions)  \
0   2015                4.50                9.57                   0.48   
1   2016                3.90              -13.26                   0.02   
2   2017                4.48               14.65                   0.18   
3   2018                4.86                8.67                   0.18   
4   2019                5.59               14.84                   0.35   
5   2020                5.98                7.12                   0.36   
6   2021                7.55               26.12                   0.65   
7   2022                8.63               14.40                   0.90   
8   2023                9.87               14.33                   1.23   
9   2024               11.31               14.61                   1.53   
10  2025               12.08                6.76                   1.62   

    Net Income Growth (%)  
0                    6.74  
1                  -95.37  
2              

## Web Collection Dataset: AlphaVantage Chipotle Stock Price API

In [31]:
def api_parser():
  # Retrieving data from Alpha Vantage API
  url = "https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY&symbol=CMG&apikey=YOUR_APLKEY"
  resp = requests.get(url)
  if resp.status_code != 200:
    print("Error: Unable to fetch data")
    return
  
  # Parsing JSON data
  data = resp.json()
  years = []
  open_prices = []
  high_prices = []
  low_prices = []
  close_prices = []
  for date, metrics in data["Monthly Time Series"].items():
    if date[:4] < "2015" or date[5:7] != "09": # Only take September data for each year from 2015 to 2025
      continue
    years.append(int(date[:4]))

    # INCONSISTENCY: Remove last two characters because cents only go to two decimal places
    open = metrics['1. open']
    open_prices.append(open[:-2])

    high = metrics['2. high']
    high_prices.append(high[:-2])

    low = metrics['3. low']
    low_prices.append(low[:-2])

    close = metrics['4. close']
    close_prices.append(close[:-2])
  
  # Creating DataFrame
  df = pd.DataFrame({'Year': years,'Open Price': open_prices,'High Price': high_prices,'Low Price': low_prices,'Close Price': close_prices})
  df.sort_values(by='Year',inplace=True, ignore_index=True)

  # Fixing Share Price to account for the 50-for-1 stock split in 2024
  df['Open Price'] = df['Open Price'].astype(float) * [1 if year < 2024 else 50 for year in df['Year']]
  df['High Price'] = df['High Price'].astype(float) * [1 if year < 2024 else 50 for year in df['Year']]
  df['Low Price'] = df['Low Price'].astype(float) * [1 if year < 2024 else 50 for year in df['Year']]
  df['Close Price'] = df['Close Price'].astype(float) * [1 if year < 2024 else 50 for year in df['Year']]


  # Writing cleaned DataFrame to CSV
  df.to_csv("cleaned_json_api.csv", index=False)
  
  print(df)



  ### EXTRA DATA NEEDED FOR MACHINE LEARNING PURPOSES ###
  months = []
  open_values = []
  high_values = []
  low_values = []
  close_values = []
  volume_values = []
  for date, metrics in data["Monthly Time Series"].items():
    if date[:4] < "2015":
        continue
    months.append(date[:-3])
    
    open = metrics['1. open']
    open_values.append(open[:-2])

    high = metrics['2. high']
    high_values.append(high[:-2])

    low = metrics['3. low']
    low_values.append(low[:-2])
    
    close = metrics['4. close']
    close_values.append(close[:-2])

    volume = metrics['5. volume']
    volume_values.append(int(volume))

  df_monthly = pd.DataFrame({'Month': months,'Open Price': open_values,'High Price': high_values,'Low Price': low_values,'Close Price': close_values,'Volume': volume_values})
  df_monthly.sort_values(by="Month",inplace=True,ignore_index=True)

  df_monthly.to_csv("cleaned_json_api_monthly.csv", index=False)

  print('\n\n')
  print(df_monthly)

############ Function Call ############
api_parser()

    Year  Open Price  High Price  Low Price  Close Price
0   2015      697.73      741.89     696.10       720.25
1   2016      415.81      441.20     395.11       423.50
2   2017      316.94      326.45     295.11       307.83
3   2018      476.19      497.87     453.67       454.52
4   2019      833.00      857.90     779.78       840.47
5   2020     1325.00     1384.46    1180.00      1243.71
6   2021     1908.00     1958.54    1816.31      1817.52
7   2022     1578.84     1754.56    1501.07      1502.76
8   2023     1928.16     1976.03    1795.01      1831.83
9   2024     2799.50     2950.00    2632.50      2881.00
10  2025     2090.50     2113.00    1915.00      1959.50



       Month Open Price High Price Low Price Close Price     Volume
0    2015-01     686.00     727.97    653.77      709.84    7857377
1    2015-02     713.55     726.63    647.28      664.97   11274541
2    2015-03     664.97     692.43    650.54      650.54    7447762
3    2015-04     650.00     699.03    619

## Insights

In [32]:
def insight1():
    df_economic = pd.read_csv("cleaned_world_economic_outlook.csv")
    df_chipotle = pd.read_csv("cleaned_html.csv")
    
    countries = df_economic['COUNTRY'].unique()
    results = []
    
    for country in countries:
        country_data = df_economic[df_economic['COUNTRY'] == country].copy()
        
        # Merge with Chipotle data
        merged = pd.merge(df_chipotle,country_data,left_on='Year',right_on='YEAR',how='inner')
        
        # Calculate correlation
        correlation = merged['Revenue (Billions)'].corr(merged['GDP (US dollars in billions)'])
        
        results.append({'Country': country,'Correlation': correlation})
    
    # Create results dataframe and sort by correlation
    results_df = pd.DataFrame(results)
    results_df = results_df.sort_values('Correlation', ascending=False, ignore_index=True)
    
    print(results_df)

############ Function Call ############
insight1()

          Country  Correlation
0   United States     0.992707
1  United Kingdom     0.952713
2          Canada     0.949468
3         Germany     0.932536
4          France     0.931466


### Insight 1 Explanation

This insight looks into the correlation between Chipotle's revenue and GDP for each of the five countries where Chipotle operates. The results show that US GDP has the strongest correlation with Chipotle's revenue, which makes sense considering approximately 95% of Chipotle locations are in the United States. France's GDP has the lowest correlation, meaning Chipotle's revenue is least influenced by France's GDP. The correlation coefficients show how closely each country's economic health relates to Chipotle's overall financial performance. This information is valuable for strategic planning because it helps Chipotle's management understand which markets drive their overall performance. It could also be used to identify opportunities for international expansion to reduce dependence on the US economy. A graphical correlation of the US GDP and Chipotle's revenue is shown in Visualization 1.

In [33]:
def insight2():
    df_economic = pd.read_csv("cleaned_world_economic_outlook.csv")
    df_chipotle = pd.read_csv("cleaned_html.csv")
    
    # Since Chipotle is primarily US-based
    us_data = df_economic[df_economic['COUNTRY'] == 'United States'].copy()

    merged = pd.merge(df_chipotle,us_data,left_on='Year',right_on='YEAR',how='inner')
    
    base_cpi = merged[merged["Year"] == 2015]['Consumer price index (CPI)'].values[0]

    df = pd.DataFrame()
    df['Year'] = merged['Year']
    df['CPI'] = merged['Consumer price index (CPI)']
    df['Nominal Income (Billions)'] = merged['Net Income (Billions)']
    df['Real Income (Billions)'] = (merged['Net Income (Billions)'] * (base_cpi / merged['Consumer price index (CPI)'])).round(2)
    df['Difference (Billions)'] = df['Nominal Income (Billions)'] - df['Real Income (Billions)']
    df['Percent Difference (%)'] = (df['Difference (Billions)'] / df['Nominal Income (Billions)'] * 100).round(2)
    print(df)

    total_nominal = df['Nominal Income (Billions)'].sum()
    total_real = df['Real Income (Billions)'].sum()
    total_diff = total_nominal - total_real
    percent_diff = total_diff / total_nominal * 100

    print(f'\nTotal Nominal Income (Billions): {total_nominal}')
    print(f'Total Real Income (Billions): {total_real}')
    print(f'Total Difference (Billions): {total_diff.round(1)}')
    print(f'Total % Difference: {percent_diff.round(2)}%')


    total_inflation = df.iloc[-1]['CPI'] / df.iloc[0]['CPI'] * 100
    dollar_value = df.iloc[0]['CPI'] / df.iloc[-1]['CPI']
    
    print(f'\nTotal Inflation: {total_inflation.round(2)}%')
    print(f'$1 in 2025 was worth ${dollar_value.round(2)} in 2015')
    

############ Function Call ############
insight2()

    Year     CPI  Nominal Income (Billions)  Real Income (Billions)  \
0   2015  237.00                       0.48                    0.48   
1   2016  240.00                       0.02                    0.02   
2   2017  245.12                       0.18                    0.17   
3   2018  251.10                       0.18                    0.17   
4   2019  255.65                       0.35                    0.32   
5   2020  258.86                       0.36                    0.33   
6   2021  270.97                       0.65                    0.57   
7   2022  292.62                       0.90                    0.73   
8   2023  304.70                       1.23                    0.96   
9   2024  313.70                       1.53                    1.16   
10  2025  323.07                       1.62                    1.19   

    Difference (Billions)  Percent Difference (%)  
0                    0.00                    0.00  
1                    0.00                  

### Insight 2 Explanation

This insight examines the difference between Chipotle's nominal (reported) net income and real net income (adjusted for inflation using the USA's CPI). Using 2015 as the base year, we calculated what each year's net income would be worth in constant 2015 dollars, revealing the true purchasing power of Chipotle's earnings. The results show that while nominal net income has grown significantly over the decade, some of the growth is due to inflation rather than actual economic gains. By comparing nominal versus real income growth, we can determine how much of Chipotle's profit increase represents actual growth rather than just inflation. This distinction is crucial to understand the company's true financial performance.

In [34]:
def insight3():
    df_stock = pd.read_csv("cleaned_json_api.csv")
    df_html = pd.read_csv("cleaned_html.csv")

    df_merged = pd.merge(df_stock, df_html, on='Year', how='inner')

    df_merged['Stock Price Growth (%)'] = (df_merged['Close Price'].pct_change() * 100).round(2)
    results_table = df_merged[['Year', 'Stock Price Growth (%)', 'Revenue Growth (%)', 'Net Income Growth (%)']]
    
    print("Year-over-Year Growth Rate Comparison:")
   
    print(results_table.to_string(index = False))

    df_valid = results_table.dropna()
    
    max_stock = df_valid["Stock Price Growth (%)"].max()
    max_rev = df_valid["Revenue Growth (%)"].max()
    max_income = df_valid["Net Income Growth (%)"].max()

    best_stock_year = df_valid[df_valid["Stock Price Growth (%)"] == max_stock].iloc[0]
    best_rev_year = df_valid[df_valid["Revenue Growth (%)"] == max_rev].iloc[0]
    best_income_year = df_valid[df_valid["Net Income Growth (%)"] == max_income].iloc[0]

    print("\nKey Takeaways:")
    print(f"Highest stock price growth: {best_stock_year['Year']} ({max_stock}%)")
    print(f"Strongest revenue growth: {best_rev_year['Year']} ({max_rev}%)")
    print(f"Best net income growth: {best_income_year['Year']} ({max_income}%)")
    

############ Function Call ############
insight3()

Year-over-Year Growth Rate Comparison:
 Year  Stock Price Growth (%)  Revenue Growth (%)  Net Income Growth (%)
 2015                     NaN                9.57                   6.74
 2016                  -41.20              -13.26                 -95.37
 2017                  -27.31               14.65                 700.00
 2018                   47.65                8.67                   0.00
 2019                   84.91               14.84                  98.86
 2020                   47.98                7.12                   1.43
 2021                   46.14               26.12                  83.66
 2022                  -17.32               14.40                  37.88
 2023                   21.90               14.33                  36.60
 2024                   57.27               14.61                  24.92
 2025                  -31.99                6.76                   5.48

Key Takeaways:
Highest stock price growth: 2019.0 (84.91%)
Strongest revenue growth:

### Insight 3 Explanation
This insight compares Chipotle's stock price growth "hype" against its revenue and profit growth "fundamentals" to see if the stock's trend matches the company's real performance. This analysis reveals that the stock is often "disconnected" from its actual sales, as investor sentiment plays a huge role. For example, in 2017, revenue grew +14.65% but the stock price fell -27.31% due to bad news. Conversely, in 2020, revenue growth was slow +7.12% but the stock jumped +47.98% as investors became hyped about its pandemic-proof app. This proves that Chipotle's stock price is driven just as much by big-picture stories as it is by its actual sales numbers.


In [35]:
def insight4():
    df_economic = pd.read_csv("cleaned_world_economic_outlook.csv")
    df_chipotle = pd.read_csv("cleaned_html.csv")

    us_data = df_economic[df_economic['COUNTRY'] == 'United States']

    df_merged = pd.merge(df_chipotle, us_data, left_on='Year', right_on='YEAR', how='inner')

    # Inflation Rate (annual % change in CPI)
    df_merged['Inflation Rate (%)'] = df_merged['Consumer price index (CPI)'].pct_change() * 100
    
    # GDP Growth Rate
    df_merged['GDP growth (percent)'] = df_merged['GDP (US dollars in billions)'].pct_change() * 100
    
    df_model_data = df_merged[[
        'Year',
        'Revenue Growth (%)',
        'GDP growth (percent)',
        'Unemployment Rate (%)',
        'Inflation Rate (%)'
    ]].copy()

    df_model_data = df_model_data.iloc[1:].reset_index(drop=True)

    y = df_model_data['Revenue Growth (%)']
    X = df_model_data[['GDP growth (percent)', 'Unemployment Rate (%)', 'Inflation Rate (%)']]


    model = LinearRegression()
    model.fit(X, y)
    
    r2 = model.score(X, y)

    coefficients = pd.DataFrame(model.coef_,X.columns,columns=['Coefficient'])


    print(f"\nPrediction Accuracy (R-squared): {round(r2, 3)}")
    print(f"{round(r2*100, 2)}% of the variance in Chipotle's Revenue Growth is explained by these 3 macro factors.")
    print("\n")
    print(coefficients)

############ Function Call ############
insight4()


Prediction Accuracy (R-squared): 0.421
42.1% of the variance in Chipotle's Revenue Growth is explained by these 3 macro factors.


                       Coefficient
GDP growth (percent)      2.612691
Unemployment Rate (%)     1.982135
Inflation Rate (%)       -0.667328


### Insight 4 Explanation

This model is an economic sensitivity analysis, not a forecasting tool. The purpose is to understand how Chipotle’s revenue growth responds to changes in GDP, unemployment, and inflation, not to predict future revenues. Because the dataset contains only a small number of annual observations, full-sample OLS is the appropriate method and is widely used in macroeconomic interpretation. The R^2 of about 42% may seem low, but it is actually strong given that only three macro factors are included while many other drivers such as store expansion, pricing strategy, marketing, consumer preferences, and supply chain events affect revenue growth. Notably, the model shows a positive sensitivity to unemployment, which may seem counterintuitive. This likely occurs because Chipotle is a value-oriented, fast-casual brand: during periods of rising unemployment, consumers often trade down from more expensive dining options toward cheaper, quick-service meals, benefiting firms like Chipotle. Finally, with only a small dataset, macro variables can be affected unrelated trends. Overall, the model finds that GDP has the strongest positive impact on revenue growth (2.61% per 1% increase), unemployment also increases revenue growth (1.98%), while inflation slightly reduces it (-0.67%).

In [36]:
def insight5():
    df = pd.read_csv("cleaned_json_api_monthly.csv")
    features = df[['Close Price', 'Volume']]

    # Normalize the data (since price and volume are on diff. scales)
    scaler = StandardScaler()
    features = scaler.fit_transform(features)

    # Applying K-means
    kmeans = KMeans(n_clusters = 4, random_state=0, n_init=10)
    df['Cluster'] = kmeans.fit_predict(features)

    stats = df.groupby('Cluster').agg({'Close Price': ['mean', 'min', 'max'], 'Volume': ['mean', 'min', 'max'], 'Month': 'count'}).round(2)
    stats.columns = ['Avg Price', 'Min Price', 'Max Price', 'Avg Volume', 'Min Volume', 'Max Volume', 'Count']
    # stats.sort_values(by='Avg Price', inplace=True)
    print(stats)
    
    cluster_labels = {
    0: "Steady Growth Period",
    1: "Pre-Split Period",
    2: "Post-Split Period",
    3: "High Growth Period"
    }

    print("\n\nClusters:")
    for num, label in cluster_labels.items():
        print(f"{num}: {label}")

    
    print("\n\nCluster Timeline Distribution:")
    for cluster_id in [0,1,2,3]:
        cluster_months = df[df['Cluster'] == cluster_id]['Month'].tolist()
        earliest = cluster_months[0]
        latest = cluster_months[-1]
        print(f"Cluster {cluster_id}: {earliest} to {latest}")



############ Function Call ############
insight5()

         Avg Price  Min Price  Max Price    Avg Volume  Min Volume  \
Cluster                                                              
0           543.66      62.65    1052.36  1.947197e+07     6555620   
1          1757.58    1155.16    3159.60  5.767548e+06     3371680   
2            43.50      31.19      56.08  4.223783e+08   336188441   
3            56.00      50.08      61.52  2.236626e+08   165561762   

         Max Volume  Count  
Cluster                     
0          84327316     67  
1           9080785     47  
2         493788343      8  
3         296086324      9  


Clusters:
0: Steady Growth Period
1: Pre-Split Period
2: Post-Split Period
3: High Growth Period


Cluster Timeline Distribution:
Cluster 0: 2015-01 to 2024-06
Cluster 1: 2020-07 to 2024-05
Cluster 2: 2024-07 to 2025-11
Cluster 3: 2024-09 to 2025-06


### Insight 5 Explanation

This insight uses K-Means clustering machine learning to identify four distinct market patterns in Chipotle's monthly stock data (2015-2025) based on closing price and number of shares. The algorithm reveals a clear progression: Cluster 0 (Steady Growth) dominated with 66 months at an average price of \$536, representing the company's core growth phase. Cluster 1 (High Growth) shows 38 months averaging \$1,542 during high-momentum periods. Cluster 3 (Pre-Split) captured 10 months before the 2024 stock split at an average of \$2,507. Most notably, Cluster 2 (Post-Split) with 17 recent months shows a much higher trading volume (312M vs. 5-20M in other clusters), demonstrating how the 50-to-1 stock split increased Chipotle's market liquidity. The current market condition (Cluster 2) indicates a post-split adjustment period with high volatility, essentially allowing this growth trend to occur again. This unsupervised learning approach successfully identified not just share prices, but fundamental shifts in trading behavior over the past decade.

## Data Visualizations

In [37]:
def visual1():
    df_economic = pd.read_csv("cleaned_world_economic_outlook.csv")
    df_chipotle = pd.read_csv("cleaned_html.csv")
    
    # Filter for US
    us_gdp = df_economic[df_economic['COUNTRY'] == 'United States'].copy()
    
    # Extract years and GDP values
    years = us_gdp['YEAR'].tolist()
    us_values = us_gdp['GDP (US dollars in billions)'].tolist()
    
    # Get Chipotle revenue data
    chipotle_years = df_chipotle['Year'].tolist()
    chipotle_revenue = df_chipotle['Revenue (Billions)'].tolist()
    
    
    # Create figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])


    fig.add_trace(
        go.Scatter(x=chipotle_years,y=chipotle_revenue,name="Chipotle Revenue"),secondary_y=False)

    fig.add_trace(go.Scatter(x=years,y=us_values,name="US GDP"),secondary_y=True)

    # Figure title
    fig.update_layout(title_text="Chipotle Revenue vs US GDP (2015-2025)")

    # x-axis title
    fig.update_xaxes(title_text="Year")

    # y-axes titles
    fig.update_yaxes(title_text="Chipotle Revenue (Billions USD)", secondary_y=False)
    fig.update_yaxes(title_text="US GDP (Billions USD)", secondary_y=True)

    fig.show()


############ Function Call ############
visual1()

### Visualization 1 Explanation

This dual y-axis line chart visualizes the relationship between Chipotle's revenue growth and the US economy (measured by GDP) from 2015-2025. The left y-axis shows Chipotle's revenue in billions of USD, while the right y-axis shows the US GDP in billions of USD. As seen in the chart, the two metrics appear to be very closely correlated. This visualization helps explain the correlation coefficient for the US calculated in Insight 1. It also allows for the correlation to be shown as a function of time to help identify any areas where the relationship may have deviated.

In [38]:
def visual2():
    
    df_economic = pd.read_csv("cleaned_world_economic_outlook.csv")
    df_chipotle = pd.read_csv("cleaned_html.csv")
    
    us_data = df_economic[df_economic['COUNTRY'] == 'United States'].copy()
    merged = pd.merge(df_chipotle,us_data,left_on='Year',right_on='YEAR',how='inner')
    
    base_cpi = merged[merged["Year"] == 2015]['Consumer price index (CPI)'].values[0]

    df = pd.DataFrame()
    df['Year'] = merged['Year']
    df['CPI'] = merged['Consumer price index (CPI)']
    df['Nominal Income (Billions)'] = merged['Net Income (Billions)']
    df['Real Income (Billions)'] = (merged['Net Income (Billions)'] * (base_cpi / merged['Consumer price index (CPI)'])).round(2)
    df['Difference (Billions)'] = df['Nominal Income (Billions)'] - df['Real Income (Billions)']
    df['Percent Difference (%)'] = (df['Difference (Billions)'] / df['Nominal Income (Billions)'] * 100).round(2)
    

    fig = go.Figure()
    
    fig.add_trace(go.Bar(
        x=df['Year'],
        y=df['Nominal Income (Billions)'],
        name='Nominal Net Income',
    ))
    
    fig.add_trace(go.Bar(
        x=df['Year'],
        y=df['Real Income (Billions)'],
        name='Real Net Income (in 2015 Dollars)',
    ))

    fig.update_layout(
        title='Nominal vs. Real Net Income (Adjusted for Inflation)',
        title_x=0.5,
        xaxis_title='Year',
        yaxis_title='Net Income (Billions)',
    )
    
    fig.show()

############ Function Call ############
visual2()

### Visualization 2 Explanation
This visualization is a Grouped Bar Chart and serves as the graphical proof for our insight on inflation's impact on profit. It plots two variables for each year: the Nominal Net Income (purple bar) to show the reported profits, and the Real Net Income (red bar), which has been adjusted for inflation using 2015 as the base year. This chart visually reveals the purchasing power of Chipotle's earnings by showing the growing gap between reported profit and its true value. For example, in the years following 2020, the Nominal Income bar shows a dramatic recovery, but the Real Income bar grows much more slowly, revealing that a significant portion of this growth is due to inflation. Therefore, this chart demonstrates that while Chipotle's profits have grown impressively on paper, a large part of that gain is an illusion created by inflation, which is crucial for understanding the company's true financial health.


In [39]:
def visual3():
    df_monthly = pd.read_csv("cleaned_json_api_monthly.csv", parse_dates=['Month'])

    july_2024 = pd.to_datetime('2024-07-01')
    post_split_mask = (df_monthly['Month'] >= july_2024)
    price_cols = ['Open Price', 'High Price', 'Low Price', 'Close Price']

    df_monthly.loc[post_split_mask, price_cols] = df_monthly.loc[post_split_mask, price_cols] * 50
    df_monthly.loc[df_monthly['Month'] == '2024-06-01', 'Low Price'] *= 50
    
    df_monthly_indexed = df_monthly.set_index('Month')

    agg_rules = {
        'Open Price': 'first',
        'High Price': 'max',
        'Low Price': 'min',
        'Close Price': 'last'
    }

    df_yearly = df_monthly_indexed.resample('Y').agg(agg_rules)
    df_yearly['Year'] = df_yearly.index.year

    df_world = pd.read_csv("cleaned_world_economic_outlook.csv")
    df_us_data = df_world[df_world['COUNTRY'] == 'United States']

    df_final = pd.merge(df_yearly, df_us_data, left_on='Year', right_on='YEAR', how='inner')
    df_final = df_final.sort_values(by='Year')

    fig = go.Figure()

    fig.add_trace(
        go.Candlestick(
            x=df_final['Year'],
            open=df_final['Open Price'],
            high=df_final['High Price'],
            low=df_final['Low Price'],
            close=df_final['Close Price'],
            name="Chipotle Stock Price"
        )
    )
    
    fig.add_trace(
        go.Scatter(
            x=df_final['Year'], 
            y=df_final['Unemployment Rate (%)'], 
            name="US Unemployment Rate (%)",
            yaxis="y2"
        )
    )

    fig.update_layout(
        title_text="Chipotle Stock Volatility vs. US Economy (2015-2025)",
        xaxis_title="Year",
        xaxis_rangeslider_visible=False,
        yaxis=dict(title="Stock Price ($)", side="left"),
        yaxis2=dict(title="US Unemployment (%)", overlaying="y", side="right")
    )

    fig.show()

############ Function Call ############
visual3()

/var/folders/gs/f8lljqh95_ld_yc0rylrm4800000gn/T/ipykernel_56566/2399165766.py:20: FutureWarning:

'Y' is deprecated and will be removed in a future version, please use 'YE' instead.



### Visualization 3 Explanation
The visualization is a Candlestick and Line Combo chart and serves as the graphical proof for insight 3. It plots two variables at once: the Stock Price (Candlesticks) to show the "hype" and the US Unemployment % (Dashed Blue Line) to show the health of the US economy. This chart visually comfirms the disconnect we found in insight 3. For example, in from 2015 to 2017 the stock price is dropping while the US economy is bettering itself, and in 2020 unemployment was at an all time high while stock price was still growing proving investors were "hyped" for the future. Therefore, the chart clearly demonstrates that investor perception and company-specific events are far more powerful drivers of the stock price than the overall US economic health, a disconnect that may not be true for other fundamental metrics like revenue or net income.